# Manually moving channels around

![star supported](https://img.shields.io/badge/STAR-supported-blue)
![Vantage supported](https://img.shields.io/badge/Vantage-supported-blue)
![OT supported](https://img.shields.io/badge/OT-supported-blue)
![EVO not tested](https://img.shields.io/badge/EVO-not%20tested-orange)

With PLR, you can easily move individual channels around. This is useful for calibrating labware locations, calibrating labware sizes, and other things.

```{warning}
Be very careful about collisions! Move channels to a safe z height before traversing.
```

```{note}
With Hamilton robots, when a tip is mounted, the z location will refer to the point of the pipetting tip. With no tip mounted, it will refer to the bottom of the channel.
```

## Example: Hamilton STAR

Here, we'll use a Hamilton STAR as an example. For other robots, simply change the deck layout, makign sure that you have at least a tip rack to use.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler, STAR
from pylabrobot.resources import STARDeck, TIP_CAR_480_A00, HTF

lh = LiquidHandler(backend=STAR(), deck=STARDeck())
await lh.setup()

# assign a tip rack
tip_carrier = TIP_CAR_480_A00(name="tip_carrier")
tip_carrier[0] = tip_rack = HTF(name="tip_rack")
lh.deck.assign_child_resource(tip_carrier, rails=0)

## Moving channels

All positions are in mm. The movements are to absolute positions. The origin will be near the left front bottom of the deck, but it differs between robots.

* x: left (low) to right (high)
* y: front (low) to back (high)
* z: bottom (low) to top (high)

In [2]:
channel = 1  # the channel to use

# start by picking up a single tip
await lh.pick_up_tips(tip_rack["A1"], use_channels=[channel])

# prepare for manual operation
# this will space the other channels to safe positions
await lh.prepare_for_manual_channel_operation(channel)

Since the channnel will be above the tip rack, it should be safe to move up. We perform a quick check to make sure the z_safe is at least above the resources we know about.

In [3]:
z_safe = 240  # WARNING: this might NOT be safe for your setup

if z_safe <= lh.deck.get_highest_known_point():
  raise ValueError(f"z_safe position is not safe, it is lower than the highest known point: {lh.deck.get_highest_known_point()}")

await lh.move_channel_z(channel, z_safe)

```{warning}
The z position in the code above should be safe for most setups, but we can't guarantee that it will be safe for all setups. Move to a z position that is above all your labware before moving in the xy plane.
```

When the z position of the bottom of the tip is above the labware, you can move the channel around in the xy plane.

In [4]:
# move the channel around
await lh.move_channel_x(channel, 100)
await lh.move_channel_y(channel, 100)

After reaching a spot where the channel can move down, you can use `move_channel_z` again.

In [ ]:
await lh.move_channel_z(channel, 100)

Before returning the tip to the tip rack, make sure to move the channel to a safe z position again.

In [ ]:
await lh.move_channel_z(channel, z_safe)

You can run the code above as often as you like. When you're done, you can return the channel to the tip rack.

In [5]:
await lh.return_tips()